In [469]:
# pour les imports
import numpy as np
import scipy.special as sp
import matplotlib.pyplot as plt
import scipy.stats as ss
from tqdm import trange
from copy import deepcopy
from tqdm import tqdm

In [465]:
n=7
m=11
points_1_10=[1,2, 3, 4, 5, 6, 7, 8, 9, 10] 
Narms=len(points_1_10)
maxPointsEnemmy=10000 # the maximum number of points of each palyer
maxPointsAgent=10000

# Zayed

In [355]:
class Cards:
    def __init__(self):
        self.ncards = (m-n+1)*4  # number of cards
        self.cards = 4*[i for i in range(n,m+1)]  # cards 

    def shuffle(self):
        '''Knuth shuffle'''
        np.random.shuffle(self.cards)

    def hands(self):
        '''distribution of cards by the agent :)'''
        self.hand1 = [self.cards[0], self.cards[2]]
        self.hand2 = [self.cards[1], self.cards[3]]



In [356]:
class Enemies :
    '''many enemies'''

    def __init__(self):
        self.maxPoints= maxPointsEnemmy # the maximum amount of points that the enemy has
        self.points= points_1_10
        
    def enemy1(self, hand1): # the enemy hand
        goforpoints = None
        if hand1[0]+hand1[1] >= 18:  # (the mean of all possible hands)
            goforpoints = np.random.choice(self.points[5:])
        else:
            goforpoints = np.random.choice(self.points[:5])
        return goforpoints
    def deterministic(self, hand1):
        sum=hand1[0]+hand1[1]
        goforpoints=None
        ## you need to add 1 for the deterministic ???????????
        if(sum==14):
            goforpoints=2
        if(sum==15):
            goforpoints=3
        if(sum==16):
             goforpoints=4
        if(sum==17):
             goforpoints=5
        if(sum==18):
             goforpoints=6
        if(sum==19):
              goforpoints=7
        if(sum==20):
              goforpoints=8
        if(sum==21):
               goforpoints=9
        if(sum==22):
            goforpoints=10
        return goforpoints
    def Human(self, hand1):
        goforpoints = None

        while not (goforpoints in self.points):
            print("your hand ", hand1)
            goforpoints = int(input("Input your action (int): ")) 

        return goforpoints
    


In [357]:
def randmax(A):
    maxValue=max(A)
    index = [i for i in range(len(A)) if A[i]==maxValue]
    return np.random.choice(index)

In [429]:
class UCB() :
    def __init__(self,alpha=0.1):
        self.alpha = alpha
        self.maxPoints = maxPointsAgent
        self.armsMu = np.zeros((Narms,2*(m-n)+1)) # P(X/s=14)=E(1X/s=14) for exemple
        self.nbDraws = np.zeros((Narms,2*(m-n)+1)) # contain the number of draws of eatch arm
        self.estimatedSumEnemyHand = None
    
    def chooseArmToGo(self,hand2,enemyResponse): # like an arm
        # sample the distrubution of the arm number enemyResponse
        if(enemyResponse==1):

            return True # always go it is a strategy
        else:
              # according to the agent technique
            if (min(self.nbDraws[enemyResponse-1,:])==0): # we explore first all the arms
                 estimatedSumEnemyHand=randmax(-self.nbDraws[enemyResponse-1,:])+14
                        
            else:
                mu=self.armsMu[enemyResponse-1,:]/self.nbDraws[enemyResponse-1,:] # the empirical mean after t-1 time
                t_1=np.sum(self.nbDraws[enemyResponse-1,:])
                sqt_alpha_log=np.sqrt(self.alpha*np.log(t_1+1)/self.nbDraws[enemyResponse-1,:])
                estimatedSumEnemyHand =randmax(mu+sqt_alpha_log)+14
           
            self.estimatedSumEnemyHand = estimatedSumEnemyHand


            if(np.sum(hand2)>=estimatedSumEnemyHand): # if the agent hand is grater the the sampled enamy hand
                return True # go for
            else :
                return False # go for -1
    def updateUCB(self, gowith,enemyResponse,hand1,hand2):
        h2=np.sum(hand2) # agent
        h1=np.sum(hand1)
        # print(type(enemyResponse),type(self.estimatedSumEnemyHand))
        if(enemyResponse !=1):
            self.nbDraws[enemyResponse-1,self.estimatedSumEnemyHand-14]+=1
            if (gowith) and h2<h1 :
                self.armsMu[enemyResponse-1,self.estimatedSumEnemyHand-14]-=enemyResponse/10
            if  h1<=h2 and not (gowith):
                self.armsMu[enemyResponse-1,self.estimatedSumEnemyHand-14]-=1/10
            if (gowith) and h2>h1:
                self.armsMu[enemyResponse-1,self.estimatedSumEnemyHand-14]+=enemyResponse/10 
            # reward == 0 if gowith and h2==h1

        

In [359]:
def mapTo1422(sampled,a14,b22):
    # int is not important
    return (b22-a14)*sampled+a14 # the distribution of an arm (a point) ?? 

In [360]:
class ThompsonSampling :
    def __init__(self,alist,blist,alpha=0.5):
        self.maxPoints= maxPointsAgent  # the maximum amount of points that the agent has
        # self.points = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        # we can keep track of this beta =>  the agent learn  ?????
        self.alist = alist
        self.blist = blist
        self.alpha = alpha

    def chooseArmToGo(self,hand2,enemyResponse): # like an arm
        # sample the distrubution of the arm number enemyResponse
        if(enemyResponse==1):

            return True # always go it is a strategy
        else:
            estimated =np.random.beta(self.alist[enemyResponse-1],self.blist[enemyResponse-1])

            estimatedSumEnemyHand=mapTo1422(estimated,14,22) # according to the agent technique
            if(np.sum(hand2)>=estimatedSumEnemyHand): # if the agent hand is grater the the sampled enamy hand
                return True # go for
            else :
                return False # go for -1
    def updateBeta(self, gowith,enemyResponse,hand1,hand2):
        h2=np.sum(hand2) # agent
        h1=np.sum(hand1)
        if(enemyResponse !=1):
            if(gowith) and h2<h1 :
                self.alist[enemyResponse-1]+=self.alpha*enemyResponse # shift bata to the right (we need to adapte the learning parameter)
            if  h1<=h2 and not (gowith):    
                self.blist[enemyResponse-1]+=self.alpha*enemyResponse # shift bata to the left
            




    

In [361]:
class Exp3 :
    def __init__(self,lr=1):
        self.maxPoints = maxPointsAgent
        self.lr = lr
        self.armsIWE = np.ones((Narms,2*(m-n)+1)) # importance-weighted estimator
        self.ListSumHands=np.arange(2*n,2*m+1)
        self.estimatedSumEnemyHand = None
    
    def chooseArmToGo(self,hand2,enemyResponse): # like an arm
        # sample the distrubution of the arm number enemyResponse
        if(enemyResponse==1):

            return True # always go it is a strategy
        else:
              # according to the agent technique
            # print(self.armsIWE[enemyResponse-1, :]/(np.sum(self.armsIWE[enemyResponse-1, :])))
            estimatedSumEnemyHand = np.random.choice(self.ListSumHands,p=self.armsIWE[enemyResponse-1, :]/(np.sum(self.armsIWE[enemyResponse-1, :])))
            self.estimatedSumEnemyHand = estimatedSumEnemyHand


            if(np.sum(hand2)>=estimatedSumEnemyHand): # if the agent hand is grater the the sampled enamy hand
                return True # go for
            else :
                return False # go for -1
    def updateExp3(self, gowith,enemyResponse,hand1,hand2):
        h2=np.sum(hand2) # agent
        h1=np.sum(hand1)
        if(enemyResponse !=1):
            if (gowith) and h2<h1 :
                self.armsIWE[enemyResponse-1,self.estimatedSumEnemyHand-14]*=np.exp(-self.lr*enemyResponse/10) 
            if  h1<=h2 and not (gowith):
                self.armsIWE[enemyResponse-1,self.estimatedSumEnemyHand-14]*=np.exp(-self.lr/10) 
            if (gowith) and h2>h1:
                self.armsIWE[enemyResponse-1,self.estimatedSumEnemyHand-14]*=np.exp(self.lr*enemyResponse/10) # it seems to be good to model the reward in (-infiny,0]

        # reward == 0 if gowith and h2==h1


    

In [362]:
def softmax(x, tau):
    x = np.array(x)
    e = np.exp(x / tau)
    return list(e / e.sum())

In [397]:
class tdLearning :

    def __init__(self, exp = .3, lr = .1, random = False, eta=2.):
        self.maxPoints= maxPointsAgent  # the maximum amount of points that the agent has
        self.points = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        # we can keep track of this beta =>  the agent learn  ?????
        self.statDict = {}
        self.exp = exp # Exploration rate
        self.lr = lr # Learning rate
        self.random = random # Si vous voulez créer une distribution
        self.eta = eta 

    def chooseArmToGo(self,hand2,enemyResponse): # like an arm
        actions = [0, 1]
        state = str(sum(hand2)) + "-" + str(enemyResponse)
        if np.random.random() < self.exp or self.statDict.get(state) is None:
            return np.random.choice(actions)
        Dict = self.statDict.get(state)
        if self.random:
            # Using softmax to help our player take "unpredectible" actions.
            # print(softmax(list(Dict.values()), self.eta))
            # print(list(Dict.keys()))
            # print(np.random.choice(list(Dict.keys()), p=softmax(list(Dict.values()), self.eta)))
            action = int(np.random.choice(list(Dict.keys()), p=softmax(list(Dict.values()), self.eta))[0])
        else:
            # Take the best action, the action who get the best reward!
            action = int(max(self.statDict.get(state), key = lambda x: self.statDict.get(state)[x]))
        return action
    
    def updateTd(self, gowith,enemyResponse,hand1,hand2):
        state = str(sum(hand2)) + "-" + str(enemyResponse)
        h2=np.sum(hand2) # agent
        h1=np.sum(hand1)

        if self.statDict.get(state) is None:
            self.statDict[state] = {}
        if self.statDict.get(state).get(str(gowith)) is None:
            self.statDict[state][str(gowith)] = 0
        
        if(gowith):
            if h1>h2 :
             self.statDict[state][str(gowith)] += self.lr*( -enemyResponse - self.statDict[state][str(gowith)])
            if h2>h1:
             self.statDict[state][str(gowith)] += self.lr*( enemyResponse - self.statDict[state][str(gowith)])
        if not gowith:    
            self.statDict[state][str(gowith)] += self.lr*( - 1 - self.statDict[state][str(gowith)])

In [364]:
# test
enemy=Enemies()
b=[1 for i in range(10)]
a=[1 for i in range(10)]
agent=UCB()
while(enemy.maxPoints>=10 and agent.maxPoints>=10):
    cards = Cards()
    cards.shuffle()
    cards.hands()
    hand1= cards.hand1
    hand2=cards.hand2
    EnemyGoFor=enemy.enemy1(hand1)
    AgentGo=agent.chooseArmToGo(hand2,EnemyGoFor)
    #show cards
    if(AgentGo):
        if(np.sum(hand1)>np.sum(hand2)):
            # agent loses
            agent.maxPoints-=EnemyGoFor
        if(np.sum(hand2)>np.sum(hand1)):
            # agent loses
            enemy.maxPoints-=EnemyGoFor
    else :
        agent.maxPoints-=1
    # update agent strategy
    agent.updateUCB(AgentGo,EnemyGoFor,hand1,hand2)
    print("=========Enemy========")
    print("hand1 ", hand1)
    print("GoFor", EnemyGoFor)
    print("maxPoints= ", enemy.maxPoints)
    print("=========Agent========")
    print("hand2 ", hand2)
    print("GoFor",AgentGo)
    print("maxPoints= ", agent.maxPoints)
if(enemy.maxPoints <10):
    print("Enemy losed")
if(agent.maxPoints <10):
    print("Agent losed")




<class 'numpy.int64'> <class 'numpy.int64'>
=========Enemy========
hand1  [11, 10]
GoFor 9
maxPoints=  100
=========Agent========
hand2  [8, 8]
GoFor True
maxPoints=  91
<class 'numpy.int64'> <class 'numpy.int64'>
=========Enemy========
hand1  [8, 9]
GoFor 4
maxPoints=  100
=========Agent========
hand2  [10, 7]
GoFor False
maxPoints=  90
<class 'numpy.int64'> <class 'numpy.int64'>
=========Enemy========
hand1  [7, 7]
GoFor 3
maxPoints=  97
=========Agent========
hand2  [7, 11]
GoFor True
maxPoints=  90
<class 'numpy.int64'> <class 'numpy.int64'>
=========Enemy========
hand1  [8, 9]
GoFor 5
maxPoints=  92
=========Agent========
hand2  [11, 11]
GoFor True
maxPoints=  90
<class 'numpy.int64'> <class 'numpy.int64'>
=========Enemy========
hand1  [8, 10]
GoFor 9
maxPoints=  92
=========Agent========
hand2  [8, 7]
GoFor False
maxPoints=  89
<class 'numpy.int64'> <class 'numpy.int64'>
=========Enemy========
hand1  [10, 8]
GoFor 9
maxPoints=  92
=========Agent========
hand2  [7, 10]
GoFor False

In [365]:
# a=agent.alist
# b=agent.blist
# x=np.linspace(0,1,50)
# x_=np.linspace(14,22,50)
# fig, axs = plt.subplots(2, 5 , figsize = (16,16))
# for i in range(0,10):
#     y = ss.beta.pdf (x, a[i], b[i],loc=0, scale=1)
#     # print(y)
#     if(i<=4):
#      h = axs[0,i].plot(x_,(22-14)*y+14,label=f"{i+1}")
#      axs[0,i].label(f"{i+1}")
#     else:
#       h = axs[1,i-5].plot(x_,(22-14)*y+14,label=f"{i+1}")
#       axs[0,i-5].label(f"{i+1}")
#     plt.legend()
# plt.subplots_adjust(wspace=0.74)
# plt.show()

In [366]:
# Beta
# a=agent.alist
# b=agent.blist
# x=np.linspace(0,1,50)
# x_=np.linspace(14,22,50)
# plt.figure(figsize=(12,8),
#            facecolor='white')
# for i in range(1,10):
#     y = ss.beta.pdf (x, a[i], b[i],loc=0, scale=1)
#     plt.plot(x_,(22-14)*y+14,label=f"{i+1}")
#     plt.legend()
# plt.xlabel("Adversary's sum hand")
# plt.ylabel("Adversary's bet")
# plt.title("The upper bound distribution for each adversary's bet (the uniform_random_mean)")
# plt.show()

In [367]:
armsIWE=agent.armsIWE

for i in range(10):
    plt.figure(figsize=(8,6),
           facecolor='white')
    plt.bar(agent.ListSumHands, height=armsIWE[i,:]/np.sum(armsIWE[i,:]),label=f"{i+1}")
    plt.legend()
    plt.show()

In [368]:
# armsIWEcopy=deepcopy(armsIWE)
# for i in range(10):
#     armsIWEcopy[i,:]=armsIWEcopy[i,:]/np.sum(armsIWEcopy[i,:])

# plt.figure(figsize=(8,8),
#            facecolor='white')
# extent = np.min(agent.ListSumHands), np.max(agent.ListSumHands),10, 2
# plt.imshow(armsIWEcopy[1:,],  cmap=plt.cm.viridis, alpha=.9, interpolation='bilinear',extent=extent)
# plt.colorbar(shrink=0.8)
# plt.xlabel("Adversary's sum hand")
# plt.ylabel("Adversary's bet")
# plt.title("The probability matrix of the Exp3 agent against the uniform_random_mean adversary")
# plt.show()

In [369]:
#agent.statDict

In [370]:

# w=agent.armsMu[1:,:]/agent.nbDraws[1:,:]
# #print(w)

# plt.figure(figsize=(8,8),
#            facecolor='white')
# extent = 14, 22,10, 2
# plt.imshow(w,  cmap=plt.cm.viridis, alpha=.9, interpolation='bilinear',extent=extent)
# plt.colorbar(shrink=0.8)
# plt.xlabel("Adversary's sum hand")
# plt.ylabel("Adversary's bet")
# plt.title("The expected reward matrix of the UCB1 agent against the uniform_random_mean adversary")
# plt.show()

In [371]:
# b=[1 for i in range(10)]
# a=[1 for i in range(10)]
# enemy=Enemies()
# agent=Exp3(0.2)
# learningtd = []

# for i in range(500):
#     score = 0
#     for j in range(100):
#         agent.maxPoints = 100
#         enemy.maxPoints = 100
#         while(enemy.maxPoints>=10 and agent.maxPoints>=10):
#             cards = Cards()
#             cards.shuffle()
#             cards.hands()
#             hand1= cards.hand1
#             hand2=cards.hand2
#             EnemyGoFor=enemy.deterministic(hand1)
#             AgentGo=agent.chooseArmToGo(hand2,EnemyGoFor)
#             #show cards
#             if(AgentGo):
#                 if(np.sum(hand1)>np.sum(hand2)):
#                     # agent loses
#                     agent.maxPoints-=EnemyGoFor
#                 if(np.sum(hand2)>np.sum(hand1)):
#                     # agent loses
#                     enemy.maxPoints-=EnemyGoFor
#             else :
#                 agent.maxPoints-=1
#             # update agent strategy
#             agent.updateIWE(AgentGo,EnemyGoFor,hand1,hand2)
#         if(enemy.maxPoints <10):
#             score += 1

#     learningtd.append(score)

In [481]:
# test
# b=[1 for i in range(10)]
# a=[1 for i in range(10)]
# enemy=Enemies()
# agent=tdLearning()
# enemyPointsList=[enemy.maxPoints]
# agentPointsList=[agent.maxPoints]
N=5000
Listrounds_gagnes=[]
Listrounds_perdus=[]
ListDelata=[]
for _ in tqdm(np.arange(N)):
    b=[1 for i in range(10)]
    a=[1 for i in range(10)]   
    enemy=Enemies()
    agent=UCB()
    Nroud=0
    while(enemy.maxPoints>=10 and agent.maxPoints>=10):
        Nroud+=1
        cards = Cards()
        cards.shuffle()
        cards.hands()
        hand1= cards.hand1
        hand2=cards.hand2
        EnemyGoFor=enemy.deterministic(hand1)
        AgentGo=agent.chooseArmToGo(hand2,EnemyGoFor)
        #show cards
        if(AgentGo):
            if(np.sum(hand1)>np.sum(hand2)):
                # agent loses
                agent.maxPoints-=EnemyGoFor
            if(np.sum(hand2)>np.sum(hand1)):
                # agent loses
                enemy.maxPoints-=EnemyGoFor
        else :
            agent.maxPoints-=1
        # agentPointsList.append(agent.maxPoints)
        # enemyPointsList.append(enemy.maxPoints)
        # update agent strategy
        agent.updateUCB(AgentGo,EnemyGoFor,hand1,hand2)
    if(enemy.maxPoints <10):
      Listrounds_gagnes.append(Nroud)
    if(agent.maxPoints <10):
      Listrounds_perdus.append(Nroud)
    ListDelata.append(agent.maxPoints-enemy.maxPoints)

100%|██████████| 5000/5000 [25:00<00:00,  3.33it/s]  


In [482]:
# print(Listrounds)
# print(ListDelata)
print(np.mean(np.array(ListDelata)/maxPointsAgent))
print("Listrounds_gagnes",len(Listrounds_gagnes))
print(np.mean(np.array(Listrounds_gagnes)))
print("Listrounds_perdus",len(Listrounds_perdus))
print(np.mean(np.array(Listrounds_perdus)))

0.6883849599999999
Listrounds_gagnes 5000
5130.5568
Listrounds_perdus 0
nan


/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# Round=np.arange(1,len(enemyPointsList)+1)
# plt.figure(figsize=(16,10),
#            facecolor='white')
# plt.plot(Round,agentPointsList, label="Agent's points")
# plt.plot(Round,enemyPointsList, label="Attacker's points")
# plt.xlabel("Number of rounds")
# plt.ylabel("Player's points")
# plt.title("TD-learning agent versus deterministic attacker")
# plt.legend()
# plt.show()